# Lane Lines from Video Stream

Detect lane lines from video stream

## Configuration

In [1]:
video_in  = 'FILE210822-173937F.mp4'
video_out = 'V5_lines.mp4'

# Parameters for predicting lane width
model_name              = 'distance'
width_prediction_height = 640

frame_width  = 1920
frame_height = 1080

mask_vertices = [
    (0, 640),
    (frame_width/2, 375),
    (frame_width, 640)
]

own_lane_vertices = [
    (500, 640),
    (int(frame_width/2), 400),
    (frame_width-500, 640)
]

left_lane_mask_top    = 375
left_lane_mask_bottom = 690
left_lane_mask_margin = 80

min_slope = 0.3
max_slope = 0

## Load and configure model

In [2]:
import os
import sys
from pathlib import Path

import cv2
import numpy as np

from tqdm.notebook import tqdm, trange


# Make sure local modules can be imported
module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)
    
# Import local modules
import cv_utils.lane_detection as lane_detection

In [3]:
# Derive paths
video_directory = os.path.join(module_path_root, 'data_sources', 'test1_video')
calibration_config = os.path.join(module_path_root, 'data_sources', 'dashcam_calibration.yml')
model_dir          = os.path.join(module_path_root, 'models')

In [4]:
# Initialise model
ld = lane_detection(calibration_config=calibration_config, model_dir=model_dir, width_estimation_model=model_name)

## Model Application

In [5]:
path_in  = os.path.join(video_directory, video_in)
path_out = os.path.join(video_directory, video_out)

cap = cv2.VideoCapture(path_in)

#frame_width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count  = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

out = cv2.VideoWriter(path_out, cv2.VideoWriter_fourcc('M','J','P','G'), 60, (frame_width, frame_height))

for frame_num in trange(0, frame_count):
    ret, frame = cap.read()
    
    if ret == True:
        image_orig = np.array(frame)

        #cv2.imwrite(os.path.join(module_path_root, 'data_sources', 'evil.png'), image_orig)
        
        image_corrected = ld.correct_image(image_orig)
        
        image_lanes, slopes_and_intercepts = ld.detect_lanes(
            image_corrected,
            own_lane_vertices,
            left_lane_mask_top    = left_lane_mask_top,
            left_lane_mask_bottom = left_lane_mask_bottom,
            left_lane_mask_margin = left_lane_mask_margin,
            min_slope             = min_slope,
            max_slope             = max_slope
        )
        
        out.write(image_lanes)
    
        cv2.imshow('object detection', cv2.resize(image_lanes, (frame_width, frame_height)))
    
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    else:
        break
        
cap.release()
out.release()
cv2.destroyAllWindows()

  0%|          | 0/3612 [00:00<?, ?it/s]